<a href="https://colab.research.google.com/github/raghuvarm17633/raghuvarm17633/blob/main/Real_Time_Chatbot_Assistanta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import tensorflow as tf
import json
import random

# Load Data
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = tf.keras.models.load_model('chatbot_model.h5')

# Preprocess Input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def get_response(ints):
    if not ints:
        return "Sorry, I don't understand."
    tag = ints[0]['intent']
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return "Sorry, I don't understand."

# Terminal-based Chatbot
def chatbot():
    print("Chatbot: Hello! Type 'exit' to quit.")
    while True:
        msg = input("You: ")
        if msg.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        ints = predict_class(msg)
        response = get_response(ints)
        print(f"Chatbot: {response}")

chatbot()


Chatbot: Hello! Type 'exit' to quit.
You: exit
Chatbot: Goodbye!
